In [6]:
!pip install hvplot jupyter_bokeh scipy cartopy intake intake-xarray dask hvplot geoviews


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.8/509.8 kB 2.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import xarray as xr
from matplotlib import pyplot as plt
import numpy as np
import hvplot.xarray
import holoviews as hv
hv.extension('bokeh')
%matplotlib inline

cat = xr.open_dataset("http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/noaa.ersst.v5/sst.mnmean.nc")

ds = cat.sel(time=slice('2000', '2018'))
ds
# size of data in MB
ds.nbytes/1e6


14.616788

In [6]:
# from google.colab import output
# output.enable_custom_widget_manager()

In [3]:

# actually download the data
ds.load()
list(ds.sst)
ds

<xarray.Dataset>
Dimensions:    (lat: 89, lon: 180, time: 228, nbnds: 2)
Coordinates:
  * lat        (lat) float32 88.0 86.0 84.0 82.0 ... -82.0 -84.0 -86.0 -88.0
  * lon        (lon) float32 0.0 2.0 4.0 6.0 8.0 ... 352.0 354.0 356.0 358.0
  * time       (time) datetime64[ns] 2000-01-01 2000-02-01 ... 2018-12-01
Dimensions without coordinates: nbnds
Data variables:
    time_bnds  (time, nbnds) float64 9.969e+36 9.969e+36 ... 9.969e+36 9.969e+36
    sst        (time, lat, lon) float32 -1.8 -1.8 -1.8 -1.8 ... nan nan nan nan
Attributes: (12/39)
    climatology:                     Climatology is based on 1971-2000 SST, X...
    description:                     In situ data: ICOADS2.5 before 2007 and ...
    keywords_vocabulary:             NASA Global Change Master Directory (GCM...
    keywords:                        Earth Science > Oceans > Ocean Temperatu...
    instrument:                      Conventional thermometers
    source_comment:                  SSTs were observed by conventional therm...
    ...                              ...
    comment:                         SSTs were observed by conventional therm...
    summary:                         ERSST.v5 is developed based on v4 after ...
    dataset_title:                   NOAA Extended Reconstructed SST V5
    _NCProperties:                   version=2,netcdf=4.6.3,hdf5=1.10.5
    data_modified:                   2023-08-03
    DODS_EXTRA.Unlimited_Dimension:  time

In [4]:
%output holomap='scrubber' fps=3
ds.sst.hvplot('lon', 'lat', cmap='Magma').redim.range(sst=(2, 40))

:DynamicMap   [time]
   :Image   [lon,lat]   (sst)

In [7]:
import cartopy.crs as ccrs
%output holomap='scrubber' fps=3
# ds.hvplot.violin(y='sst', by='time.year')
# ds.hvplot.heatmap(x='lon', y='lat', C='sst')
# ds.sst.mean(dim=['lat', 'lon']).hvplot(by='time.year', groupby=['time.year'])


ds.hvplot.quadmesh(
    'lon', 'lat', 'sst', projection=ccrs.Orthographic(-90, 30),
    global_extent=True, frame_height=540, cmap='viridis',
    coastline=True
)

/Users/spencerromberg/Repos/ocean_data/venv/lib/python3.9/site-packages/cartopy/io/__init__.py:241: DownloadWarning: Downloading: https://naturalearth.s3.amazonaws.com/110m_physical/ne_110m_coastline.zip
  warnings.warn(f'Downloading: {url}', DownloadWarning)


:DynamicMap   [time]
   :Overlay
      .QuadMesh.I  :QuadMesh   [lon,lat]   (sst)
      .Coastline.I :Feature   [Longitude,Latitude]

In [ ]:
# from google.colab import output
# output.disable_custom_widget_manager()